In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr

# 0D Plot Metrics

The plot metrics we previously created can be 0D (single value), 1D (vertical profile), 2D (single values across an xy grid) or 3D (vertical profiles across an xy grid - ie. voxels). 

At the moment, to compare plots we just want 0D metrics.

For the non gridded metrics, we've already calculated some summary stats over the vertical profiles, so we can just drop those metrics / data_vars. Similarly, we can drop the vertical profiles in the gridded metrics to just have 2D metrics. Now we can summarise across the x and y dimensions using something like sd or cv to capture variability. 

In [6]:
data_dir = Path("../data")
plots_dir = data_dir / "outputs" / "plots"
metrics_no_grid_dir = plots_dir / "metrics" / "no_grid_z_1m"

no_grid_files = [p for p in metrics_no_grid_dir.glob("*.nc")]

In [7]:
def drop_z_data_vars(ds):
    vars_without_z = [v for v in ds.data_vars if 'z' not in ds[v].dims]
    ds = ds[vars_without_z]
    return ds

In [8]:
no_grid_plot_metrics = xr.open_mfdataset(no_grid_files, parallel=True, combine='nested', concat_dim='plot')


In [9]:
no_grid_plot_metrics = drop_z_data_vars(no_grid_plot_metrics)
no_grid_plot_metrics_df = no_grid_plot_metrics.stack(plot_variant=("plot", "variant")).to_dataframe()
no_grid_plot_metrics_df

max    min      range       mean     median  \
plot        variant                                                      
EPO_O_04_P2 default  53.980999  0.000  53.980999  21.851965  10.820000   
            rnw      53.980999  0.000  53.980999  31.263412  40.191002   
            fr       53.980999  0.000  53.980999  21.851965  10.820000   
            v0       53.980999  0.001  53.980000  23.649569  13.101000   
EPY_Y_03_P4 default  16.204000  0.000  16.204000   4.428008   3.172000   
...                        ...    ...        ...        ...        ...   
ULM_80_P1   v0       49.824001  0.001  49.823002  17.971943  13.538000   
ULM_345_P4  default  63.140999  0.000  63.140999  22.728144  18.993000   
            rnw      63.140999  0.000  63.140999  27.562914  25.232000   
            fr       63.140999  0.000  63.140999  22.728144  18.993000   
            v0       63.140999  0.001  63.139999  23.465954  19.679001   

                            sd         var        cv       crr      skew  ...  \
plot        variant                                                       ...   
EPO_O_04_P2 default  17.769808  315.766052  0.813190  0.404808  0.277927  ...   
            rnw      16.258944  264.353271  0.520063  0.579156 -0.600343  ...   
            fr       17.769808  315.766052  0.813190  0.404808  0.277927  ...   
            v0       17.298246  299.229309  0.731440  0.438099  0.179021  ...   
EPY_Y_03_P4 default   4.400423   19.363722  0.993770  0.273266  0.427730  ...   
...                        ...         ...       ...       ...       ...  ...   
ULM_80_P1   v0       12.784046  163.431824  0.711334  0.360696  0.654579  ...   
ULM_345_P4  default  15.424580  237.917648  0.678655  0.359959  0.538931  ...   
            rnw      15.062350  226.874405  0.546472  0.436530  0.353307  ...   
            fr       15.424580  237.917648  0.678655  0.359959  0.538931  ...   
            v0       15.110514  228.327606  0.643933  0.371634  0.555715  ...   

                          fhd  norm_fhd  cv_inside  cv_inside_p    cv_ppi  \
plot        variant                                                         
EPO_O_04_P2 default  3.439175  0.858220   1.144310     1.144310  0.065998   
            rnw      3.403595  0.849342   1.182717     1.182717  0.124094   
            fr       3.414188  0.851985   1.160770     1.160770  0.071305   
            v0       3.431055  0.860133   1.135970     1.135970  0.105402   
EPY_Y_03_P4 default  2.368286  0.819371   1.272887     1.272887  0.045085   
...                       ...       ...        ...          ...       ...   
ULM_80_P1   v0       3.704267  0.946893   0.652030     0.652030  0.086278   
ULM_345_P4  default  3.971260  0.951340   0.623735     0.623735  0.078449   
            rnw      4.004307  0.959256   0.562694     0.562694  0.098922   
            fr       3.952137  0.946759   0.668497     0.668497  0.079889   
            v0       3.955923  0.951198   0.624875     0.624875  0.060842   

                       cv_vad      site  site_type         plot  variant  
plot        variant                                                       
EPO_O_04_P2 default  1.472094  EPO_O_04        EPO  EPO_O_04_P2  default  
            rnw      1.487138  EPO_O_04        EPO  EPO_O_04_P2      rnw  
            fr       1.476632  EPO_O_04        EPO  EPO_O_04_P2       fr  
            v0       1.644352  EPO_O_04        EPO  EPO_O_04_P2       v0  
EPY_Y_03_P4 default  0.699263  EPY_Y_03        EPY  EPY_Y_03_P4  default  
...                       ...       ...        ...          ...      ...  
ULM_80_P1   v0       1.410220    ULM_80        ULM    ULM_80_P1       v0  
ULM_345_P4  default  1.858045   ULM_345        ULM   ULM_345_P4  default  
            rnw      1.572866   ULM_345        ULM   ULM_345_P4      rnw  
            fr       1.857127   ULM_345        ULM   ULM_345_P4       fr  
            v0       1.222139   ULM_345        ULM   ULM_345_P4       v0  

[1260 rows x 39 columns]

In [10]:
no_grid_plot_metrics.close()

In [18]:
no_grid_plot_metrics_df['grid_size'] = 0
no_grid_plot_metrics_df['grid_summary'] = None
no_grid_plot_metrics_df['plot_number'] = no_grid_plot_metrics_df['plot'].apply(lambda p: int(p[-1:]))
no_grid_plot_metrics_df

max    min      range       mean     median  \
plot        variant                                                      
EPO_O_04_P2 default  53.980999  0.000  53.980999  21.851965  10.820000   
            rnw      53.980999  0.000  53.980999  31.263412  40.191002   
            fr       53.980999  0.000  53.980999  21.851965  10.820000   
            v0       53.980999  0.001  53.980000  23.649569  13.101000   
EPY_Y_03_P4 default  16.204000  0.000  16.204000   4.428008   3.172000   
...                        ...    ...        ...        ...        ...   
ULM_80_P1   v0       49.824001  0.001  49.823002  17.971943  13.538000   
ULM_345_P4  default  63.140999  0.000  63.140999  22.728144  18.993000   
            rnw      63.140999  0.000  63.140999  27.562914  25.232000   
            fr       63.140999  0.000  63.140999  22.728144  18.993000   
            v0       63.140999  0.001  63.139999  23.465954  19.679001   

                            sd         var        cv       crr      skew  ...  \
plot        variant                                                       ...   
EPO_O_04_P2 default  17.769808  315.766052  0.813190  0.404808  0.277927  ...   
            rnw      16.258944  264.353271  0.520063  0.579156 -0.600343  ...   
            fr       17.769808  315.766052  0.813190  0.404808  0.277927  ...   
            v0       17.298246  299.229309  0.731440  0.438099  0.179021  ...   
EPY_Y_03_P4 default   4.400423   19.363722  0.993770  0.273266  0.427730  ...   
...                        ...         ...       ...       ...       ...  ...   
ULM_80_P1   v0       12.784046  163.431824  0.711334  0.360696  0.654579  ...   
ULM_345_P4  default  15.424580  237.917648  0.678655  0.359959  0.538931  ...   
            rnw      15.062350  226.874405  0.546472  0.436530  0.353307  ...   
            fr       15.424580  237.917648  0.678655  0.359959  0.538931  ...   
            v0       15.110514  228.327606  0.643933  0.371634  0.555715  ...   

                     cv_inside_p    cv_ppi    cv_vad      site  site_type  \
plot        variant                                                         
EPO_O_04_P2 default     1.144310  0.065998  1.472094  EPO_O_04        EPO   
            rnw         1.182717  0.124094  1.487138  EPO_O_04        EPO   
            fr          1.160770  0.071305  1.476632  EPO_O_04        EPO   
            v0          1.135970  0.105402  1.644352  EPO_O_04        EPO   
EPY_Y_03_P4 default     1.272887  0.045085  0.699263  EPY_Y_03        EPY   
...                          ...       ...       ...       ...        ...   
ULM_80_P1   v0          0.652030  0.086278  1.410220    ULM_80        ULM   
ULM_345_P4  default     0.623735  0.078449  1.858045   ULM_345        ULM   
            rnw         0.562694  0.098922  1.572866   ULM_345        ULM   
            fr          0.668497  0.079889  1.857127   ULM_345        ULM   
            v0          0.624875  0.060842  1.222139   ULM_345        ULM   

                            plot  variant  grid_size  grid_summary  \
plot        variant                                                  
EPO_O_04_P2 default  EPO_O_04_P2  default          0          None   
            rnw      EPO_O_04_P2      rnw          0          None   
            fr       EPO_O_04_P2       fr          0          None   
            v0       EPO_O_04_P2       v0          0          None   
EPY_Y_03_P4 default  EPY_Y_03_P4  default          0          None   
...                          ...      ...        ...           ...   
ULM_80_P1   v0         ULM_80_P1       v0          0          None   
ULM_345_P4  default   ULM_345_P4  default          0          None   
            rnw       ULM_345_P4      rnw          0          None   
            fr        ULM_345_P4       fr          0          None   
            v0        ULM_345_P4       v0          0          None   

                     plot_number  
plot        variant               
EPO_O_04_P2 default        

In [19]:
metrics_10m_grid_dir = plots_dir / "metrics" / "grid_10m_z_1m"
grid_10m_files = [p for p in metrics_10m_grid_dir.glob("*.nc")]

In [20]:
plot_metrics_grid_10m = xr.open_mfdataset(grid_10m_files, parallel=True, combine='nested', concat_dim='plot')

/Users/jgregory/Code/forest-structure-tools/.pixi/envs/default/lib/python3.13/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 152
  result = blockwise(
/Users/jgregory/Code/forest-structure-tools/.pixi/envs/default/lib/python3.13/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 152
  result = blockwise(
/Users/jgregory/Code/forest-structure-tools/.pixi/envs/default/lib/python3.13/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 152
  result = blockwise(
/Users/jgregory/Code/forest-structure-tools/.pixi/envs/default/lib/python3.13/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 152
  result = blockwise(
/Users/jgregory/Code/forest-structure-tools/.pixi/envs/default/lib/python3.13/site-packages/dask/array/core.py:5003: PerformanceWarning: Increasing number of chunks by factor of 152
  

KeyboardInterrupt: 